# 05 Quantile forecasting using Gradient Boosted Trees

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import constants as cnst
import stock_utils as su

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
QUANTILE_LB, QUANTILE_UB = 0.1, 0.9

stock_symbols = su.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT', 'HDFCBANK', 'INDIGOPNTS', 'ITBEES', 'JUBLFOOD']

In [4]:
STOCK_SYMBOL = stock_symbols[1]
STOCK_SYMBOL

'HDFCBANK'

## Loading stock data

In [5]:
stock_data = su.StockData(STOCK_SYMBOL)
stock_data

Symbol: HDFCBANK
Total records: 1049
First record: 2020-01-01
Last record: 2024-02-26

## Modelling

### Target columns

In [6]:
target_cols = stock_data.standardized.filter(regex = "Target.*").columns.to_list()
target_cols

['Target 3D', 'Target 7D', 'Target 15D', 'Target 30D']

In [7]:
pred_input_df = stock_data.standardized.drop(columns = target_cols)
pred_input_df.shape

(1049, 43)

In [8]:
stock_data.standardized[target_cols].describe()

,Target 3D,Target 7D,Target 15D,Target 30D
count,1046.000000,1042.000000,1034.000000,1019.000000
mean,1.000772,1.001874,1.003761,1.008427
std,0.030943,0.046347,0.065198,0.088580
min,0.820000,0.718000,0.668000,0.629000
25%,0.985000,0.977000,0.967000,0.959500
50%,1.000000,1.002000,1.001500,1.006000
75%,1.016750,1.025000,1.042000,1.058000
max,1.178000,1.219000,1.231000,1.322000


### Data processing

In [9]:
def get_training_data(target_col: str):
    print(f"Target: {target_col}")
    X_df = stock_data.standardized[
        stock_data.standardized[target_col].notnull()
    ].drop(columns = target_cols).copy()
    y = stock_data.standardized[
        stock_data.standardized[target_col].notnull()
    ][target_col].copy()
    print(f"X.shape: {X_df.shape}")
    print(f"y.shape: {y.shape}")

    return X_df, y

### Grid search parameters

In [10]:
param_dict = {
    "n_estimators": [100, 125, 150],
    "learning_rate": [0.05, 0.1, 0.2],
    "max_depth": [2, 3, 4],
    "max_features": ["log2", "sqrt", 0.25],
    "subsample": [0.75, 1.0]
}

feature_importances = pd.DataFrame(
    index = stock_data.standardized.drop(columns = target_cols).columns.to_list()
)

### Model building

In [11]:
def get_model():
    gb_model = GradientBoostingRegressor(
        loss = "squared_error",
        random_state = cnst.RANDOM_STATE
    )

    grid_cv = RandomizedSearchCV(
        gb_model,
        param_dict,
        n_iter = 12,
        cv = 5,
        n_jobs = -1,
        random_state = cnst.RANDOM_STATE
    )

    return grid_cv

def get_quantile_model(quantile: float):
    gb_model = GradientBoostingRegressor(
        loss = "quantile",
        alpha = quantile,
        random_state = cnst.RANDOM_STATE
    )

    grid_cv = RandomizedSearchCV(
        gb_model,
        param_dict,
        n_iter = 12,
        cv = 5,
        n_jobs = -1,
        random_state = cnst.RANDOM_STATE
    )

    return grid_cv

def print_results(y, preds):
    print(f"Target std: {y.std():.3f}")
    print(f"R2: {r2_score(y, preds):.3f}")
    print(f"MSE: {mean_squared_error(y, preds, squared = False):.3f}")
    print(f"MAE: {mean_absolute_error(y, preds):.3f}")

## Training

### `Target 3D`

#### Forecasting model

In [12]:
target_col = target_cols[0]
X, y = get_training_data(target_col)

Target: Target 3D
X.shape: (1046, 43)
y.shape: (1046,)


In [13]:
model = get_model()
model.fit(X, y)
model.best_params_

{'subsample': 0.75,
 'n_estimators': 100,
 'max_features': 'sqrt',
 'max_depth': 3,
 'learning_rate': 0.05}

In [14]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score').iloc[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_features,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.218195,0.006510,0.003628,0.000251,0.75,100,sqrt,3,0.05,"{'subsample': 0.75, 'n_estimators': 100, 'max_...",-0.073567,-0.125814,-0.061809,-0.041317,-0.151693,-0.090840,0.041279,1
2,0.306997,0.036363,0.005763,0.001548,1.0,100,log2,4,0.05,"{'subsample': 1.0, 'n_estimators': 100, 'max_f...",-0.223753,-0.161684,-0.099582,-0.026423,-0.190555,-0.140399,0.070104,2
4,0.234855,0.016141,0.004583,0.000985,1.0,125,sqrt,2,0.1,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.293165,-0.129442,-0.067115,-0.078092,-0.178879,-0.149339,0.082209,3
11,0.305156,0.015473,0.002404,0.000117,0.75,150,sqrt,4,0.05,"{'subsample': 0.75, 'n_estimators': 150, 'max_...",-0.111340,-0.209215,-0.093146,-0.099336,-0.390107,-0.180629,0.112926,4
8,0.350782,0.013772,0.003843,0.000684,1.0,125,sqrt,4,0.05,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.243895,-0.197180,-0.124212,-0.063272,-0.317459,-0.189204,0.088980,5


In [15]:
feature_importances = feature_importances.join(
    pd.DataFrame(
        model.best_estimator_.feature_importances_,
        index = model.best_estimator_.feature_names_in_,
        columns = [target_col]
    )
)

preds = model.predict(X)
print_results(y, preds)

Target std: 0.031
R2: 0.417
MSE: 0.024
MAE: 0.018


#### Lower quantile model

In [16]:
lb_model = get_quantile_model(QUANTILE_LB)
lb_model.fit(X, y)
lb_model.best_params_

{'subsample': 0.75,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Upper quantile model

In [17]:
ub_model = get_quantile_model(QUANTILE_UB)
ub_model.fit(X, y)
ub_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Predictions

In [18]:
pred_col_name = f'Pred {target_col}'
lb_col_name, ub_col_name = f'LB {target_col}', f'UB {target_col}'

stock_data.processed[pred_col_name] = (model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[lb_col_name] = (lb_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[ub_col_name] = (ub_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)

stock_data.processed[['Date', 'Close', target_col, pred_col_name, lb_col_name, ub_col_name]].dropna().iloc[-10:, :]

,Date,Close,Target 3D,Pred Target 3D,LB Target 3D,UB Target 3D
1036,2024-02-08,1403.05,1394.45,1402.61,1382.20,1439.17
1037,2024-02-09,1403.60,1384.05,1404.47,1371.81,1447.08
1038,2024-02-12,1390.00,1414.05,1403.97,1361.25,1417.27
1039,2024-02-13,1394.45,1419.90,1402.77,1361.87,1429.96
1040,2024-02-14,1384.05,1417.10,1388.50,1363.84,1417.64
1041,2024-02-15,1414.05,1454.30,1423.98,1381.58,1453.78
1042,2024-02-16,1419.90,1439.15,1421.19,1384.80,1445.48
1043,2024-02-19,1417.10,1419.55,1420.20,1387.28,1441.47
1044,2024-02-20,1454.30,1420.60,1447.83,1416.47,1487.21
1045,2024-02-21,1439.15,1422.30,1436.98,1402.17,1466.64


### `Target 7D`

#### Forecasting model

In [19]:
target_col = target_cols[1]
X, y = get_training_data(target_col)

Target: Target 7D
X.shape: (1042, 43)
y.shape: (1042,)


In [20]:
model = get_model()
model.fit(X, y)
model.best_params_

{'subsample': 0.75,
 'n_estimators': 100,
 'max_features': 'sqrt',
 'max_depth': 3,
 'learning_rate': 0.05}

In [21]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score').iloc[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_features,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.248578,0.011002,0.004023,0.000714,0.75,100,sqrt,3,0.05,"{'subsample': 0.75, 'n_estimators': 100, 'max_...",-0.184783,-0.320680,-0.156486,-0.204813,-0.130674,-0.199487,0.065599,1
2,0.341342,0.067782,0.006913,0.001933,1.0,100,log2,4,0.05,"{'subsample': 1.0, 'n_estimators': 100, 'max_f...",-0.231562,-0.488055,-0.279517,-0.136296,-0.119479,-0.250982,0.132545,2
8,0.413541,0.007780,0.004874,0.000753,1.0,125,sqrt,4,0.05,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.294381,-0.661652,-0.205458,-0.181840,-0.099330,-0.288532,0.196639,3
11,0.366555,0.009094,0.003754,0.000714,0.75,150,sqrt,4,0.05,"{'subsample': 0.75, 'n_estimators': 150, 'max_...",-0.281948,-0.543452,-0.294790,-0.242357,-0.122493,-0.297008,0.137411,4
4,0.358930,0.023915,0.004589,0.000523,1.0,125,sqrt,2,0.1,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.276380,-0.691950,-0.200647,-0.215827,-0.229221,-0.322805,0.186305,5


In [22]:
feature_importances = feature_importances.join(
    pd.DataFrame(
        model.best_estimator_.feature_importances_,
        index = model.best_estimator_.feature_names_in_,
        columns = [target_col]
    )
)

preds = model.predict(X)
print_results(y, preds)

Target std: 0.046
R2: 0.511
MSE: 0.032
MAE: 0.025


#### Lower quantile model

In [23]:
lb_model = get_quantile_model(QUANTILE_LB)
lb_model.fit(X, y)
lb_model.best_params_

{'subsample': 0.75,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Upper quantile model

In [24]:
ub_model = get_quantile_model(QUANTILE_UB)
ub_model.fit(X, y)
ub_model.best_params_

{'subsample': 0.75,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Predictions

In [25]:
pred_col_name = f'Pred {target_col}'
lb_col_name, ub_col_name = f'LB {target_col}', f'UB {target_col}'

stock_data.processed[pred_col_name] = (model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[lb_col_name] = (lb_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[ub_col_name] = (ub_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)

stock_data.processed[['Date', 'Close', target_col, pred_col_name, lb_col_name, ub_col_name]].dropna().iloc[-10:, :]

,Date,Close,Target 7D,Pred Target 7D,LB Target 7D,UB Target 7D
1032,2024-02-02,1446.15,1394.45,1424.02,1393.40,1458.45
1033,2024-02-05,1444.85,1384.05,1424.42,1386.52,1456.75
1034,2024-02-06,1444.10,1414.05,1428.02,1409.39,1476.03
1035,2024-02-07,1429.95,1419.90,1430.08,1408.25,1473.81
1036,2024-02-08,1403.05,1417.10,1420.77,1392.55,1444.34
1037,2024-02-09,1403.60,1454.30,1424.94,1395.59,1453.26
1038,2024-02-12,1390.00,1439.15,1394.09,1363.11,1439.47
1039,2024-02-13,1394.45,1419.55,1407.65,1378.10,1442.29
1040,2024-02-14,1384.05,1420.60,1399.15,1383.52,1430.87
1041,2024-02-15,1414.05,1422.30,1409.26,1384.80,1431.48


### `Target 15D`

#### Forecasting model

In [26]:
target_col = target_cols[2]
X, y = get_training_data(target_col)

Target: Target 15D
X.shape: (1034, 43)
y.shape: (1034,)


In [27]:
model = get_model()
model.fit(X, y)
model.best_params_

{'subsample': 1.0,
 'n_estimators': 100,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.05}

In [28]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score').iloc[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_features,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,0.442409,0.074137,0.005193,0.001215,1.0,100,log2,4,0.05,"{'subsample': 1.0, 'n_estimators': 100, 'max_f...",-0.061978,-0.649951,-0.388411,-0.569792,-0.366464,-0.407319,0.203251,1
10,0.254165,0.003964,0.003962,0.000860,0.75,100,sqrt,3,0.05,"{'subsample': 0.75, 'n_estimators': 100, 'max_...",-0.070894,-0.540832,-0.371051,-0.473566,-0.585761,-0.408421,0.183601,2
8,0.450533,0.019187,0.005112,0.000659,1.0,125,sqrt,4,0.05,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.100836,-0.481098,-0.596756,-0.660922,-0.682443,-0.504411,0.213605,3
4,0.361127,0.042502,0.009772,0.007322,1.0,125,sqrt,2,0.1,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.125389,-0.944257,-0.407470,-0.418737,-0.626732,-0.504517,0.271596,4
11,0.345902,0.027628,0.002945,0.000496,0.75,150,sqrt,4,0.05,"{'subsample': 0.75, 'n_estimators': 150, 'max_...",-0.041374,-0.771890,-0.624026,-0.477835,-0.608280,-0.504681,0.249700,5


In [29]:
feature_importances = feature_importances.join(
    pd.DataFrame(
        model.best_estimator_.feature_importances_,
        index = model.best_estimator_.feature_names_in_,
        columns = [target_col]
    )
)

preds = model.predict(X)
print_results(y, preds)

Target std: 0.065
R2: 0.794
MSE: 0.030
MAE: 0.023


#### Lower quantile model

In [30]:
lb_model = get_quantile_model(QUANTILE_LB)
lb_model.fit(X, y)
lb_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Upper quantile model

In [31]:
ub_model = get_quantile_model(QUANTILE_UB)
ub_model.fit(X, y)
ub_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Predictions

In [32]:
pred_col_name = f'Pred {target_col}'
lb_col_name, ub_col_name = f'LB {target_col}', f'UB {target_col}'

stock_data.processed[pred_col_name] = (model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[lb_col_name] = (lb_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[ub_col_name] = (ub_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)

stock_data.processed[['Date', 'Close', target_col, pred_col_name, lb_col_name, ub_col_name]].dropna().iloc[-10:, :]

,Date,Close,Target 15D,Pred Target 15D,LB Target 15D,UB Target 15D
1024,2024-01-20,1478.85,1394.45,1422.91,1395.93,1514.10
1025,2024-01-23,1427.35,1384.05,1379.34,1383.29,1395.11
1026,2024-01-24,1455.90,1414.05,1440.20,1398.95,1500.62
1027,2024-01-25,1434.90,1419.90,1429.78,1375.24,1478.37
1028,2024-01-29,1454.65,1417.10,1435.45,1404.55,1514.11
1029,2024-01-30,1444.30,1454.30,1440.05,1385.00,1494.66
1030,2024-01-31,1462.55,1439.15,1446.89,1406.22,1523.53
1031,2024-02-01,1466.35,1419.55,1436.22,1404.54,1494.44
1032,2024-02-02,1446.15,1420.60,1427.74,1394.82,1492.36
1033,2024-02-05,1444.85,1422.30,1425.66,1391.74,1491.72


### `Target 30D`

#### Forecasting model

In [33]:
target_col = target_cols[3]
X, y = get_training_data(target_col)

Target: Target 30D
X.shape: (1019, 43)
y.shape: (1019,)


In [34]:
model = get_model()
model.fit(X, y)
model.best_params_

{'subsample': 1.0,
 'n_estimators': 100,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.05}

In [35]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score').iloc[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_features,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,0.349954,0.050060,0.005258,0.001068,1.0,100,log2,4,0.05,"{'subsample': 1.0, 'n_estimators': 100, 'max_f...",-0.067345,-0.238801,-1.026120,-0.729262,-0.195820,-0.451470,0.364837,1
10,0.266586,0.014334,0.003998,0.000749,0.75,100,sqrt,3,0.05,"{'subsample': 0.75, 'n_estimators': 100, 'max_...",-0.010371,-0.472030,-1.011686,-0.697731,-0.255242,-0.489412,0.346637,2
5,0.943887,0.058013,0.005129,0.000416,1.0,150,0.25,4,0.1,"{'subsample': 1.0, 'n_estimators': 150, 'max_f...",0.033925,0.006566,-1.437104,-0.631014,-0.487378,-0.503001,0.536121,3
0,0.466548,0.049439,0.006735,0.001441,0.75,150,log2,4,0.1,"{'subsample': 0.75, 'n_estimators': 150, 'max_...",-0.063007,-0.316956,-1.420828,-0.341010,-0.430358,-0.514432,0.469357,4
9,0.476536,0.031475,0.005144,0.001352,1.0,150,log2,4,0.1,"{'subsample': 1.0, 'n_estimators': 150, 'max_f...",-0.131173,-0.539115,-1.298372,-0.573429,-0.259954,-0.560409,0.404902,5


In [36]:
feature_importances = feature_importances.join(
    pd.DataFrame(
        model.best_estimator_.feature_importances_,
        index = model.best_estimator_.feature_names_in_,
        columns = [target_col]
    )
)

preds = model.predict(X)
print_results(y, preds)

Target std: 0.089
R2: 0.879
MSE: 0.031
MAE: 0.024


#### Lower quantile model

In [37]:
lb_model = get_quantile_model(QUANTILE_LB)
lb_model.fit(X, y)
lb_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 0.25,
 'max_depth': 4,
 'learning_rate': 0.1}

#### Upper quantile model

In [38]:
ub_model = get_quantile_model(QUANTILE_UB)
ub_model.fit(X, y)
ub_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Predictions

In [39]:
pred_col_name = f'Pred {target_col}'
lb_col_name, ub_col_name = f'LB {target_col}', f'UB {target_col}'

stock_data.processed[pred_col_name] = (model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[lb_col_name] = (lb_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[ub_col_name] = (ub_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)

stock_data.processed[['Date', 'Close', target_col, pred_col_name, lb_col_name, ub_col_name]].dropna().iloc[-10:, :]

,Date,Close,Target 30D,Pred Target 30D,LB Target 30D,UB Target 30D
1009,2024-01-01,1698.10,1394.45,1429.64,1389.00,1757.96
1010,2024-01-02,1699.10,1384.05,1426.98,1380.29,1751.40
1011,2024-01-03,1672.90,1414.05,1434.56,1373.80,1700.29
1012,2024-01-04,1690.85,1419.90,1438.21,1380.90,1748.41
1013,2024-01-05,1682.20,1417.10,1459.89,1379.85,1737.91
1014,2024-01-08,1663.45,1454.30,1459.68,1420.40,1711.67
1015,2024-01-09,1650.50,1439.15,1480.95,1439.52,1699.26
1016,2024-01-10,1655.95,1419.55,1465.33,1419.45,1719.34
1017,2024-01-11,1649.00,1420.60,1464.24,1419.93,1698.82
1018,2024-01-12,1641.20,1422.30,1457.91,1392.23,1686.07


## Feature importances

In [40]:
feature_importances['Mean'] = feature_importances.mean(axis = 1)
(feature_importances.sort_values('Mean', ascending = False) * 100).round(1)

,Target 3D,Target 7D,Target 15D,Target 30D,Mean
52W H,8.6,7.9,8.6,10.1,8.8
PE,4.7,7.7,7.0,10.6,7.5
DayOfYear,3.5,7.5,8.1,9.4,7.1
Range 60MA,3.5,8.4,5.9,9.1,6.7
52W L,2.1,5.1,7.0,7.5,5.4
Month,1.3,2.6,7.2,7.3,4.6
Range 7MA,3.4,6.3,4.1,2.4,4.0
Range 30MA,3.5,3.4,4.4,4.5,3.9
Close 60MA,4.2,3.2,5.1,3.0,3.9
Year,0.7,2.1,4.8,7.1,3.7


## Forecasts

In [41]:
stock_data.processed.filter(regex = "(Date)|(Close$)|(Pred.*)").iloc[-10:, :]

,Date,Close,Pred Target 3D,Pred Target 7D,Pred Target 15D,Pred Target 30D
1039,2024-02-13,1394.45,1402.77,1407.65,1385.55,1387.16
1040,2024-02-14,1384.05,1388.50,1399.15,1374.59,1341.86
1041,2024-02-15,1414.05,1423.98,1409.26,1381.29,1383.65
1042,2024-02-16,1419.90,1421.19,1401.46,1373.00,1391.97
1043,2024-02-19,1417.10,1420.20,1399.82,1365.51,1401.23
1044,2024-02-20,1454.30,1447.83,1429.88,1427.78,1439.27
1045,2024-02-21,1439.15,1436.98,1426.47,1400.99,1425.21
1046,2024-02-22,1419.55,1426.08,1417.77,1377.48,1418.19
1047,2024-02-23,1420.60,1426.74,1406.88,1359.15,1413.27
1048,2024-02-26,1422.30,1431.20,1413.16,1360.64,1415.99


In [42]:
latest_preds = stock_data.processed.iloc[-1]
print(f"Date: {latest_preds['Date'].date()}")
print(f"Close: {latest_preds['Close']}")

for target_col in target_cols:
    print(f"{target_col}: {latest_preds[f'Pred {target_col}']}", end = " ")
    print(f"({latest_preds[f'LB {target_col}']} to {latest_preds[f'UB {target_col}']})")

Date: 2024-02-26
Close: 1422.3
Target 3D: 1431.2 (1380.66 to 1453.27)
Target 7D: 1413.16 (1341.84 to 1463.12)
Target 15D: 1360.64 (1359.37 to 1499.81)
Target 30D: 1415.99 (1296.51 to 1547.85)
